In [3]:
import numpy as np
import torch 
import os
import sys
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint
from typing import *
sys.path.append('./..')
from PanjivaDataPreprocessor import data_fetcher_v1

In [21]:
'''
Set the MASK token to be 0
token weights = 0 - means the loss does not need to be accounted for 
'''
def get_maksed_tokens(
    data, 
    column_cardinalities, 
    k1 = 0.8, 
    k2 = 0.2,
    uniform_column_choice = True
):
    column_indices = np.array(data)
    mask_token_id = 0

    # =======
    num_attr = data.shape[-1]
    if uniform_column_choice:
        column_prob = np.array([1/num_attr for _ in range(num_attr)])
    else:
        column_prob = np.array([np.power(column_cardinalities[i]/sum(column_cardinalities),0.5) for i in range(num_attr)]) 
        column_prob = column_prob/np.sum(column_prob)
    print('[Columns Probability]',column_prob)    
    
    inp_mask = np.random.multinomial(1, column_prob, column_indices.shape[:1])
    inp_mask = np.array(inp_mask, dtype=bool)
    print('column_indices.shape', column_indices.shape)
    # ======
    # Set targets to -1 by default, it means ignore
    # the labels to be predicted 
    # ======

    labels = -1 * np.ones(column_indices.shape, dtype=int)
    labels[inp_mask] = column_indices[inp_mask]

    column_indices_masked = np.copy(column_indices)
    # Replace k1 fraction of tokens with [MASK] following MLM
    
    inp_mask_2mask = inp_mask & (np.random.rand(*column_indices_masked.shape) < k1)
    column_indices_masked[inp_mask_2mask] = mask_token_id  

    # The indices that are not going to be changed
    # Set this fraction to k2
   
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*column_indices_masked.shape) < k2)
    
    # ----------------------
    # Replace masks in each column by randomly sampling from candidate values
    # ----------------------
    values = []
    for i in range(num_attr):
        _values = np.random.randint(1, column_cardinalities[i]+1, inp_mask_2random[:,i].sum())
        print(_values)
        values.extend(_values)
        
    column_indices_masked[inp_mask_2random] =  values
    data_masked = column_indices_masked
    token_weights = np.ones(labels.shape)
    token_weights[labels == -1] = 0

    return data_masked, labels, token_weights

'''
Sample run
'''
# column_indices = [
#     [12,90,64,24, 240],
#     [22,91,65,25,241],
#     [32,92,66,26,242],
#     [42,93,67,27,243]
# ]
# column_indices = np.array(column_indices)
# column_cardinalities = [45,100,70,30,250]

# get_maksed_tokens(
#     column_indices, 
#     column_cardinalities,
#     uniform_column_choice = False
# )

'\nSample run\n'

In [25]:
data = data_fetcher_v1.get_training_set_data('us_import1')
del data['PanjivaRecordID']
data = data.head(1000)
data = data.values
domain_dims_df = data_fetcher_v1.get_domain_dims('us_import1')
cardinality = domain_dims_df['dimension'].tolist()

In [26]:
cardinality

[531, 2385, 1546, 263, 60, 126, 84, 2158]

In [27]:
get_maksed_tokens(
    data, 
    cardinality
)

[Columns Probability] [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
column_indices.shape (1000, 8)
[403 289 106 152  15 141 291  82 145 485 197 342 385  72 112 461 170  52
 506 412 125 175 166 177 147 228 185]
[1196 1524 1428 1366 1748 1862  795 1456  935 1148  483 1061  335  274
  430 2285  666]
[ 135  132 1151  353 1141 1184 1308  181   15   79  229  906 1339  291
  145   98  364  781  479  560  561 1306  781  471  553  642 1202]
[170  90 145 258  51 140  11 241  78 108 146  85 108 110 261]
[39 37 42 52 31 35 37 45 45 20 18 13 47 60  6  5 33]
[121  55  34 117 106  89   2  37   9  81 125  65   8  90 113  73  52  31
  61  91]
[44 67 39 40 46 62 50 79  9 47 47 53 19 31  4 65 45]
[1054  330 1445 1786   76 1663 1269 1987  766   17  437 1275 1881  629]


(array([[ 206, 1054, 1113, ...,   82,   15, 1242],
        [  79,  237,  529, ...,   83,   15,  403],
        [ 529, 2369, 1210, ...,   20,   15,  933],
        ...,
        [ 207, 2039,  821, ...,   92,   34,  710],
        [ 140,  543,  583, ...,    0,   15, 2107],
        [   0,  749,  955, ...,   36,   80, 1953]]),
 array([[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1, 1777],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   10,   -1,   -1],
        [ 299,   -1,   -1, ...,   -1,   -1,   -1]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]))